In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import pandas as pd

data = pd.read_csv("/content/drive/MyDrive/star_classification.csv", sep=',')
output = data['class']
input = data.drop(['class','rerun_ID'], axis=1)

input_max = input.max()
input_min = input.min()
input_normalized = (input - input_min) / (input_max - input_min)
df = pd.DataFrame(input_normalized)

# # Compute the correlation matrix
correlation_matrix = df.corr()

# # Visualize the correlation matrix using a heatmap
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.show()
dataset = pd.concat([input_normalized, output], axis=1)

# Shuffle the dataset (by shuffling rows)
shuffled_dataset = dataset.sample(frac=1, random_state=42).reset_index(drop=True)

# Calculate the index to split the dataset (90% for training and 10% for testing)
train_size = int(0.9 * len(shuffled_dataset))

# Split the dataset into training and testing sets
train_data = shuffled_dataset[:train_size]
test_data = shuffled_dataset[train_size:]

# Separate features and labels for training and testing
input_normalized = train_data.drop('class', axis=1)
output = train_data['class']

X_test = test_data.drop('class', axis=1)
Y_test = test_data['class']

import matplotlib.pyplot as plt

# Plot histogram of the classes
output.value_counts().plot(kind='bar', color='skyblue')
plt.title('Class Distribution Before Oversampling Minority Cases')
plt.xlabel('Classes')
plt.ylabel('Frequency')
plt.show()

# Check class distribution
class_counts = output.value_counts()
print(class_counts)

max_class_size = class_counts.max()
classes = class_counts.index

print(f"Majority Class Size: {max_class_size}")

# oversample
oversampled_data = []
# Loop through each class
for cls in classes:
    class_data = input_normalized[output == cls]  # Get data for the current class
    oversampled_class = class_data.sample(max_class_size, replace=True, random_state=42)  # Oversample with replacement
    oversampled_data.append(oversampled_class)

# Combine all oversampled classes
oversampled_input = pd.concat(oversampled_data, axis=0)

# Create the corresponding labels
oversampled_output = pd.Series(
    [cls for cls in classes for _ in range(max_class_size)],
    name='class'
)


#shuffle
# Reset indices of oversampled data and labels
oversampled_input = oversampled_input.reset_index(drop=True)
oversampled_output = oversampled_output.reset_index(drop=True)

# Combine input and output
balanced_data = pd.concat([oversampled_input, oversampled_output], axis=1)

# Shuffle the dataset
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)
input = data.drop(['class'], axis=1)

data = balanced_data

output = data['class']

input = data.drop(['class'], axis=1)


input_max = input.max()

input_min = input.min()

input_normalized = (input - input_min) / (input_max - input_min)

import matplotlib.pyplot as plt


balanced_data['class'].value_counts().plot(kind='bar', color='lightblue', edgecolor='black')
plt.title('Class Distribution After Oversampling')
plt.xlabel('Classes')
plt.ylabel('Frequency')
plt.xticks(rotation=0)
plt.show()

In [ ]:


def one_hot_representation(class_array):
    data = list(set(class_array))
    num_classes = len(data)
    one_hot = np.eye(num_classes)
    one_hot_encoded = np.array([one_hot[:,data.index(label)] for label in class_array])

    # one=np.where(class_array == data[0], one_hot[:,2] , 0)
    label = np.split(one_hot,one_hot.shape[1],axis=0)

    return one_hot_encoded , label

def loss(y_true, y_pred):

    epsilon = 1e-9  # Small value to prevent log(0)
    # Clip y_pred to avoid log(0)
    y_pred = np.clip(y_pred, epsilon, 1. - epsilon)

    # Compute cross-entropy loss

    # print(y_pred)
    loss = -np.sum(y_true * np.log(y_pred)) / y_true.shape[0]
    return loss
def softmax(z):
    # Subtracting the max of z for numerical stability
    # This prevents overflow by scaling the logits

    z_exp = np.exp(z - np.max(z, axis=1, keepdims=True))  # Exponentiate the logits (with stability adjustment)
    # print(z - np.max(z, axis=1, keepdims=True))
    # print(np.sum(z_exp, axis=1, keepdims=True))
    return z_exp / np.sum(z_exp, axis=0, keepdims=True)  # Normalize the values to sum to 1
def accuracy_score(y_test, y_pred):
    # Check if the lengths of y_test and y_pred are the same
    if len(y_test) != len(y_pred):
        raise ValueError("The length of y_test and y_pred must be the same.")
    y_test_class = np.argmax(y_test, axis=1)
    y_pred_class = np.argmax(y_pred, axis=1)
    # Count the number of correct predictions (where y_test[i] == y_pred[i])
    correct_predictions = np.sum(y_test_class == y_pred_class)

    # Calculate accuracy
    accuracy = correct_predictions / len(y_test)

    return accuracy
def f1(y_test, y_pred, class_labels):
    # Initialize F1 scores list for each class
    f1_scores = []

    # For each class label
    for label in class_labels:
        # True Positives (TP): Correct predictions of the current class
        TP = np.sum((y_test == label) & (y_pred == label))
        # False Positives (FP): Incorrect predictions of the current class (predicted as 'label' but actually not 'label')
        FP = np.sum((y_test != label) & (y_pred == label))
        # False Negatives (FN): Instances of the current class that were misclassified
        FN = np.sum((y_test == label) & (y_pred != label))

        # Precision and Recall
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0

        # F1 Score for this class
        f1_class = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        f1_scores.append(f1_class)

    # Return the average F1 score (macro average in this case)
    return np.mean(f1_scores)
class MultiClassSVM:
    def _init_(self, classes = 3, regularization = 0, epoch = 5, learning_rate = 0.001, kernel="linear"):
        self.classifiers = {}
        self.class_indices = {}  # Map class tuples to integer indices
        self.num_classes = classes
        self.regularization = regularization
        self.epoch = epoch
        self.lr = learning_rate
        self.kernel = kernel
    def fit(self, X, Y):

        unique_classes = np.unique(Y, axis=0)

        for i in range(len(unique_classes)):
            for j in range(i + 1, len(unique_classes)):

                relevant_indices = (Y == unique_classes[i]) | (Y == unique_classes[j])

                relevant = np.where(np.all(relevant_indices, axis=1))

                input = X[relevant[0]]
                output = Y[relevant[0]]

                binary_labels = np.where(np.all(output == unique_classes[i], axis=1), -1, 1)

                model = SVM(learning_rate=self.lr, epoch= self.epoch, num_classes=self.num_classes, regularization=self.regularization, kernel = self.kernel)


                model.fit(input, binary_labels)

                class_tuple = (tuple(unique_classes[i]), tuple(unique_classes[j]))
                self.classifiers[class_tuple] = model

                # Create mapping for class tuple to index
                if class_tuple not in self.class_indices:
                    self.class_indices[class_tuple] = (i, j)

    def predict(self, data):
        # Determine the number of classes
        #num_classes = len(self.classifiers) + 1  # Assuming one vs. one classification

        # Initialize votes array with dimensions (num_samples, num_classes)
        votes = np.zeros((len(data), self.num_classes))
        class_dict = {}

        for (class1, class2), classifier in self.classifiers.items():


            # Get predictions for the binary classifier
            predictions = classifier.predict(data)

            # Map binary predictions (-1, 1) to votes for respective classes
            class1_index, class2_index = self.class_indices[(class1, class2)]



            class_dict[str(class1_index)] = class1
            class_dict[str(class2_index)] = class2
            votes[:, class1_index] += np.where(predictions == -1, 1, 0).flatten()  # Votes for class1
            votes[:, class2_index] += np.where(predictions == 1, 1, 0).flatten()   # Votes for class2

        # Determine the class with the highest votes for each sample

        pred_indices = np.argmax(votes, axis=1)
        predictions = []
        for i in range(len(pred_indices)):

            z = class_dict[str(pred_indices[i])]

            predictions.append(z)


        return predictions




class SVM:
    def __init__(self, learning_rate=0.001, epoch=10, num_classes=3, kernel = "linear", regularization=0, gamma = 0, degree = 2, coef0 = 1):
        self.lr = learning_rate
        self.epoch = epoch
        self.classes = num_classes
        self.parameters={}
        self.losses = {}
        self.kernel = kernel
        self.regularization = regularization
        self.gamma = gamma
        self.degree = degree
        self.coef0 = coef0
    def kernel_function(self, X, X2=None):
        if self.kernel == 'linear':
            return X
        elif self.kernel == 'rbf':
    
            sq_dists = np.sum(X*2, axis=1).reshape(-1, 1) + np.sum(X2*2, axis=1) - 2 * np.dot(X, X2.T)
            return np.exp(-self.gamma * sq_dists)
        elif self.kernel == 'polynomial':
            return (np.dot(X, X2.T) + self.coef0) ** self.degree
        else:
            raise ValueError("Kernel not supported")

    def forward_pass(self, X):
        # Use the kernel function to calculate the decision function
        K = self.kernel_function(X)
        z = np.dot(K, self.parameters["W"]) + self.parameters["b"]
        A = softmax(z)
        return A

    def fit(self, X, y):

        # Call kernel function

        X = self.kernel_function(X,X)
        n_samples, n_features = X.shape


        # init parameters
        # for i in range(n_features):

        self.parameters["W"] = np.random.randn(n_features,1) * 0.01  # Weight matrix
        self.parameters["b"] = np.zeros((1,1))  # Bias vector

        # gradient descent
        for _ in range(self.epoch):
            for j in range(n_samples):

                A = X[j].reshape(n_features,1)

                prediction =(np.dot(A.T, self.parameters["W"]) + self.parameters["b"])

                margin = y[j]* prediction
                # print((np.dot(X[j]., self.parameters["W"])).shape)
                if margin >= 1:
                    dw = 2 * self.regularization * self.parameters["W"]
                    db = 0
                else:
                    dw = 2 * self.regularization * self.parameters["W"] - y[j] * A
                    db = y[j]
                self.parameters["W"] = self.parameters["W"] - self.lr * dw
                self.parameters["b"] = self.parameters["b"] - self.lr * db

    def predict(self, X):


        X = self.kernel_function(X,X)
        y_hat = np.dot(X, self.parameters["W"]) + self.parameters["b"]
        A = np.sign(y_hat)

        return A
        
def compute_confusion_matrix(y_test , y_pred):

    Y_pred_classes = np.argmax(y_pred, axis=1)
    Y_test_classes = np.argmax(y_test, axis=1)

    # Create a confusion matrix
    conf_matrix = confusion_matrix(Y_test_classes, Y_pred_classes)

# Display the confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=["Galaxy", "Star", "QSO"])
    disp.plot(cmap="Blues")
    plt.title("Confusion Matrix for SVM")
    plt.show()

    return conf_matrix
def k_fold(X, Y, k=5, shuffle=False,learning = 0.001,epoch = 1000, regularization = 0):
    # Step 1: Shuffle the data
    if shuffle:
        indices = np.random.permutation(len(X))

        X_shuffled = X.iloc[indices]
        Y_shuffled = Y.iloc[indices]


    else:
        X_shuffled = X
        Y_shuffled = Y
    Y_shuffled_ohr, oh_labels = one_hot_representation(Y_shuffled) # One hot representation of classes

    # Step 2: Split data into k folds
    fold_size = len(X) // k
    accuracies = []  # List to store accuracy for each fold
    f1_scores = []
    cm = []
    for i in range(k):
    # Step 3: Define the test set for this fold
        test_start = i * fold_size
        test_end = (i + 1) * fold_size if i != k - 1 else len(X)

    # Split data into training and test sets
        X_train = np.concatenate([X_shuffled[:test_start], X_shuffled[test_end:]], axis=0)
        Y_train = np.concatenate([Y_shuffled_ohr[:test_start], Y_shuffled_ohr[test_end:]], axis=0)
        X_valid = X_shuffled[test_start:test_end]
        Y_valid = Y_shuffled_ohr[test_start:test_end]


        # Step 4: Train the model on the training set
        model = MultiClassSVM(classes = len(oh_labels),regularization= regularization, epoch=epoch,)
        # model(learning , iterations , num_class)
        model.fit(X_train, Y_train)

        # Step 5: Make predictions on the test set
        Y_pred = model.predict(X_valid)

        # Step 6: Calculate accuracy for this fold
        accuracy = accuracy_score(Y_valid, Y_pred)
        accuracies.append(accuracy)
        f1_score = f1(Y_valid, Y_pred, oh_labels)
        f1_scores.append(f1_score)
    # Step 7: Calculate the average accuracy across all folds
    average_accuracy = np.mean(accuracies)
    average_f1_score = np.mean(f1_scores)
    print("For SVM: ")
    print("For learning rate " + str(learning) + " and epoch " + str(epoch)  + " and regularization " + str(regularization) )
    print("Validation Accuracy: " + str(average_accuracy))
    print("Validation F1 Score: " + str(average_f1_score) + "\n")
    return accuracies, average_accuracy, f1_scores, average_f1_score,model




In [ ]:
Y_test_ohr, oh_lab = one_hot_representation(Y_test)

Acc, Av_acc, F1, Av_f1, model = k_fold(input_normalized, output,k=5, shuffle=True,learning = 0.001 ,epoch=70,regularization = 0)
print("Validation Accuracies are " + str(Acc))
print("Validation F1 Scores are " + str(F1))

best = np.argmax(F1)
best_model = model[best]

predictions = best_model.predict(X_test)
accuracy = accuracy_score(Y_test_ohr, predictions)
f1_score = f1(Y_test_ohr, predictions, oh_lab)
cm = (compute_confusion_matrix(Y_test_ohr, predictions))
print("Test Accuracy: " + str(accuracy))
print("Test F1 Score: " + str(f1_score))